In [3]:
import numpy as np
import json
import matplotlib as plt
# Waypoint altitude step : 35ft=10.668m , 200ft=60.96m , 1000ft=304.8m





def Cal_patherror(WP, X):
    path = np.array(WP[0]) - np.array(WP[1])
    A = np.array(X[1:4]) - np.array(WP[1])
    B = np.array(WP[0]) - np.array(X[1:4])

    s = np.linalg.norm(path)  # a+b
    a = np.linalg.norm(A)
    b = np.linalg.norm(B)

    q1 = (a**2 - b**2) / s  # a-b
    q2 = (s + q1) / 2  # a

    distance = abs(np.sqrt(a**2 - q2**2))  # Distance deviated from the original path
    return distance


# Waypoint altitude step : 35ft=10.668m , 200ft=60.96m , 1000ft=304.8m

def runge_kutta_4th_order(x, deriv, save_state, sum_k, t, Num, dt):
    deriv = np.array(deriv)
    if Num == 1:
        save_state = x
        sum_k = deriv
        x = save_state + deriv / 2 * dt
        t = t + dt/2
        Num += 1
    elif Num == 2:
        sum_k = sum_k + deriv*2
        x = save_state + deriv / 2 * dt
        Num += 1
    elif Num == 3:
        sum_k = sum_k + deriv*2
        x = save_state + deriv * dt
        t = t + dt/2
        Num += 1
    elif Num == 4:
        sum_k = sum_k + deriv
        x = save_state + dt/6 * sum_k
        Num += 1
    else:
        print('d')

    return x, t, save_state, sum_k, Num

# Note: In Req_Vh, the check for 'j < size(UAV.WP, 1)' is changed to 'j < len(UAV["WP"]) - 1'
# Because python list indices start at 0, we subtract 1 from the length of the list to get the last index.

def Req_Vh(Req, UAV,  j, X, WP):
    WP1 = UAV["WP"][j]
    WP2 = UAV["WP"][j+1]
    Req[3] = np.sqrt((WP[1][0]-X[1])**2 + (WP[1][1]-X[2])**2)  # R(4) = Req_del_dis (x,y distance)

    if X[6] != 0:
        t_go = Req[3] / X[6]
    else:
        t_go = float('inf')  # or any other value that makes sense in your context
    t_go_time = 5

    if j < len(UAV["WP"]) - 1:
        if np.linalg.norm(np.array(WP2[2])-np.array(WP1[2])) == 0:  # Mission execution
            Req[1] = UAV["Max_V"][j][1]
        else:
            Req[1] = 0  # During landing
    else:
        Req[1] = 0
    return Req

# Note: In Req_psi, we use np.arctan2 instead of np.arccos to find the angle between two vectors.

def Req_psi(Req, X, WP):
    psi = X[4]  # UAV_psi
    WP0 = [X[1], X[2], 0]  # initial x,y position
    WP1 = [X[1]+np.cos(psi), X[2]+np.sin(psi), 0]  # x,y position vector1
    WP2 = [WP[1][0], WP[1][1], 0]  # x,y target
    A = np.array(WP1) - np.array(WP0)
    B = np.array(WP2) - np.array(WP0)

    if np.linalg.norm(B) > 1:  # Point not arrived
        if abs(WP[1][2]-WP[0][2]) == 0 and Req[0] == 0:  # Horizontal motion
            del_psi = np.arctan2(np.linalg.norm(np.cross(A, B)), np.dot(A, B))
        else:  # Vertical motion
            del_psi = 0
    else:
        del_psi = 0

    Req[2] = del_psi  # R(3) = Req_del_psi
    return Req

# Note: In UPDATE_DERIV, 'Req_z = WP[1][2]' is changed to 'Req_z = WP[1][2]'
# because python list indices start at 0

def UPDATE_DERIV(Req, X, WP, SPEC):
    Req_z   = WP[1][2]  # target height
    Req_Vv  = Req[0]
    Req_Vh  = Req[1]
    Del_psi = Req[2]
    Del_dis = Req[3]

    t   = X[0]
    x   = X[1]
    y   = X[2]
    z   = X[3]
    psi = X[4]  # radian
    Vv  = X[5]
    Vh  = X[6]
    bat = X[13]

    d_x   = Vh * np.cos(psi)  # true dx
    d_y   = Vh * np.sin(psi)  # true dy
    d_z   = Vv

    if Req_Vv == 0:
        psi = np.arctan2(d_y, d_x)

    K = [1.4, 1.3, 0.01, 0.75, 3, 0.1, 0.1, 0.1, 0.015, 0.015]
    # | 1    2  | 3    4 |  5  |  6    7    8     9    10
    # |  Vertical V |  Horizontal V  | Psi |         Battery        |

    d_Vv  = 8*(Req_Vv-Vv)

    d_Vh  = 4*(Req_Vh-Vh)
    if d_Vh > SPEC["dVh_max"]:
        d_Vh = SPEC["dVh_max"]
    elif d_Vh < -SPEC["dVh_max"]:
        d_Vh = -SPEC["dVh_max"]

    d_psi = 4*(Del_psi)
    if d_psi > SPEC["dpsi_max"]:
        d_psi = SPEC["dpsi_max"]
    elif d_psi < -SPEC["dpsi_max"]:
        d_psi = -SPEC["dpsi_max"]

    d_bat = -K[5]*abs(d_Vv) - K[6]*abs(d_Vh) - K[7]*abs(d_psi) - K[8]*(abs(Vv)) - K[9]*(abs(Vh))  # cost function A

    x      = [x, y, z, Vv, Vh, psi, bat]
    deriv  = [d_x, d_y, d_z, d_Vv, d_Vh, d_psi, d_bat]
    return t, x, deriv


def UPDATE_STATE(X, x, deriv, t, Req, WP, Wind, SPEC):
    D2R = np.pi / 180
    R2D = 180 / np.pi

    Windmag = Wind[0]
    Winddir = Wind[1] * D2R
    d_x_w = Windmag * np.cos(-Winddir - 90 * D2R)  # wind dx
    d_y_w = Windmag * np.sin(-Winddir - 90 * D2R)  # wind dy

    D_x = x[4] * np.cos(x[5]) - d_x_w
    D_y = x[4] * np.sin(x[5]) - d_y_w
    D_z = x[3]

    V_r = [D_x, D_y, D_z]

    if np.linalg.norm(np.array(WP[0]) - np.array(WP[1])) == 0:
        vect = np.zeros(3)
    else:
        vect = (np.array(WP[1]) - np.array(WP[0])) / np.linalg.norm(np.array(WP[1]) - np.array(WP[0]))

    AOA_theta = np.degrees(np.arctan2(SPEC["W"], 0.5 * SPEC["rho"] * np.linalg.norm(V_r) ** 2 * SPEC["S"] * SPEC["Cd"]))
    Thrust = np.sqrt(SPEC["W"] ** 2 + (0.5 * SPEC["rho"] * np.linalg.norm(V_r) ** 2 * SPEC["S"] * SPEC["Cd"]) ** 2)

    AOA_psi = np.rad2deg((np.arctan2(D_y, D_x) + 2 * np.pi) % (2 * np.pi))

    X[0] = t  # t
    X[1] = x[0]  # x
    X[2] = x[1]  # y
    X[3] = x[2]  # z
    X[4] = x[5]  # psi
    X[5] = x[3]  # Vv
    X[6] = x[4]  # Vh
    X[7] = deriv[0]
    X[8] = deriv[1]
    X[9] = deriv[2]
    X[10] = deriv[5]
    X[11] = deriv[3]
    X[12] = deriv[4]
    X[13] = x[6]
    X[14] = AOA_psi
    X[15] = AOA_theta
    X[16] = Thrust

    Req = Req_psi(Req, X, WP)  # [Vertical speed, Horizontal speed, angle, distance]
    return X, Req


# Load data
with open('C:\Users\leeyj\lab_ws\data', 'r') as f:
    data = json.load(f)
Path_1 = data['Path_1']

UAV = []  # list of dictionaries, each representing a UAV
DATA = []  # list of dictionaries, each representing data for a UAV
J = []  # list of dictionaries, each representing J for a UAV
X = []  # list of lists, each representing X for a UAV
f_path = [{} for _ in Path_1]  # Initialize with empty dictionaries
VReq = [{} for _ in Path_1]  # Initialize with empty dictionaries
Xx = []

# Initial settings
t = 0
dt = 0.1
velocity = 9.33  # velocity setting
Vv_up = 6
Vv_down = -6
alt = 30

# Now you can assign values to the keys
for path in Path_1:
    UAV.append({"WP": path["Optimal1"]})
    DATA.append({"S": [], "D": []})
    J.append({"length": list(range(1, len(path["Optimal1"]) + 1))})

for i in range(len(UAV)):
    # Create f_path
    f_path[i]["final"] = np.array(UAV[i]["WP"])[:, 0:2].tolist()

    # Assign start and end points to UAV
    UAV[i]["start"] = f_path[i]["final"][0]
    UAV[i]["end"] = f_path[i]["final"][-1]

    # Add altitude to the waypoints
    f_path[i]["final"] = [[*coord, alt] for coord in f_path[i]["final"]]
    WayP = f_path[i]["final"]

    # Create UAV waypoints
    UAV[i]["WP"] = [[WayP[0][0], WayP[0][1], 0]] + WayP + [[WayP[-1][0], WayP[-1][1], 0]]

    # Create velocity requirements
    VReq[i]["V"] = np.zeros((len(UAV[i]["WP"]) - 1, 4))
    VReq[i]["V"][0, 0] = Vv_up
    VReq[i]["V"][-1, 0] = Vv_down
    VReq[i]["V"][1:-1, 1] = velocity
    UAV[i]["Max_V"] = VReq[i]["V"]

for i in range(len(UAV)):
    UAV[i]['Pos'] = UAV[i]['WP'][0]
    UAV[i]['t'] = 0
    UAV[i]['bat'] = 100
    UAV[i]['vel'] = 7
    vert = np.array(UAV[i]['WP'][2][:2]) - np.array(UAV[i]['WP'][1][:2])  # initial psi direction
    UAV[i]['psi'] = np.arctan2(vert[1], vert[0])  # [rad]
    Xx.append([UAV[i]['t'], *UAV[i]['Pos'], UAV[i]['psi'], 0, 0, 0, 0, 0, 0, 0, 0, 0, UAV[i]['bat'], np.rad2deg(UAV[i]['psi']), 90, 0])
    # [t x y z psi Vv Vh dx dy dz dpsi dVv dVh bat, aoa_psi, aoa_theta Thrust]
    #  1 2 3 4  5  6  7  8  9  10  11  12  13  14   15      16   17
    DATA[i]['S'] = Xx[i]
    J[i]['length'] = list(range(1, len(UAV[i]['WP']) + 1))
    J[i]['length'][-1] = 0


# Define SPEC
SPEC = {
    "Weight": 5,
    "W": 5 * 9.807,
    "S": 0.827,
    "Cd": 0.96,
    "rho": 1.225,
    "dpsi_max": np.pi / 4,
    "dVv_max": 6 / 2,
    "dVh_max": 9.33 / 2
}

# Initialize
Wind = [0, 0]
dt = 0.1


# Initialize UAVs
for i in range(len(UAV)):
    UAV[i]['Pos'] = UAV[i]['WP'][0]
    UAV[i]['t'] = 0
    UAV[i]['bat'] = 100
    vert = [np.array(UAV[i]['WP'][2][0:2]) - np.array(UAV[i]['WP'][1][0:2]) for i in
            range(len(UAV))]  # Initial psi direction
    UAV[i]['psi'] = np.arctan2(vert[i][1], vert[i][0])  # [rad]
    Xx = [[UAV[i]['t'], *UAV[i]['Pos'], UAV[i]['psi'], 0, 0, 0, 0, 0, 0, 0, 0, UAV[i]['bat'], np.rad2deg(UAV[i]['psi']), 90, 0] for i in range(len(UAV))]
    DATA[i]['S'] = Xx[i]
    J[i]['length'] = list(range(1, len(UAV[i]['WP']) + 1))
    J[i]['length'][-1] = 0

# Path planning process
j = [1] * len(UAV)  # Update waypoint each UAV
while True:
    n_count = 1
    Num = 1
    save_state = [0] * len(Xx[0])
    sum_k = [0] * len(Xx[0])
    for i in range(len(UAV)):
        if j[i] != 0:
            WP = [UAV[i]['WP'][j[i] - 1], UAV[i]['WP'][j[i]]]
            X = Xx[i]
            dist = Cal_patherror(WP, X)

            Req = VReq[i]['V'][j[i] - 1]
            Req = Req_Vh(Req, UAV[i], j[i], X, WP)
            Req = Req_psi(Req, X, WP)
            Req_z = abs(WP[1][2] - X[3])

            while Num < 5:
                t, x, deriv = UPDATE_DERIV(Req, X, WP, SPEC)
                x, t, save_state, sum_k, Num = runge_kutta_4th_order(x, deriv, save_state, sum_k, t, Num, dt)
                X, Req = UPDATE_STATE(X, x, deriv, t, Req, WP, Wind, SPEC)

            Num = 1
            save_state[n_count] = save_state
            n_count += 1

            DATA[i]['S'].append(X)
            DATA[i]['D'].append(dist)


            if Req_z <= 1 and j[i] == 1:  # Ascend
                j[i] = J[i]['length'][j[i]]
            elif Req[3] <= 2 and Req_z <= 1 and j[i] != 1:  # Normal
                j[i] = J[i]['length'][j[i]]
            elif X[3] <= 0 and j[i] == J[i]['length'][-2]:  # Landing
                j[i] = J[i]['length'][j[i]]

    if all(val == 0 for val in j):
        break


# Plotting
for i in range(len(UAV)):
    plt.plot(np.array(DATA[i]["S"])[:, 1], np.array(DATA[i]["S"])[:, 2])
plt.show()

#.. 3D simulation graph
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in range(len(UAV)):
    ax.plot(DATA[i].S[:,1], DATA[i].S[:,2], DATA[i].S[:,3], '.', markersize=4)
ax.set_title('3D simulation')
ax.set_zlim([0, 50])
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.set_zlabel('Alt [m]')
plt.show()

#.. Horizontal Velocity
plt.figure()
for i in range(len(UAV)):
    plt.plot(DATA[i].S[:,0], DATA[i].S[:,6], '.-', markersize=4)
plt.grid(True)
plt.xlabel('Time [sec]')
plt.ylabel('Vh [m/s]')
plt.show()

#.. Thrust Vector graph (psi)
plt.figure()
for i in range(len(UAV)):
    plt.plot(DATA[i].S[:,0], DATA[i].S[:,14], '.-', markersize=4)
plt.grid(True)
plt.xlabel('Time [sec]')
plt.ylabel('psi_T [Deg]')
plt.show()

#.. Thrust Vector graph (theta)
plt.figure()
for i in range(len(UAV)):
    plt.plot(DATA[i].S[:,0], DATA[i].S[:,15], '.-', markersize=4)
plt.grid(True)
plt.xlabel('Time [sec]')
plt.ylabel('theta_T [Deg]')
plt.show()

#.. d_psi
plt.figure()
for i in range(len(UAV)):
    plt.plot(DATA[i].S[:,0], DATA[i].S[:,10], '.-', markersize=4)
plt.grid(True)
plt.xlabel('Time [sec]')
plt.ylabel('d_psi [Rad/s]')
plt.show()

#.. d_Vertical Velocity
plt.figure()
for i in range(len(UAV)):
    plt.plot(DATA[i].S[:,0], DATA[i].S[:,11], '.-', markersize=4)
plt.grid(True)
plt.xlabel('Time [sec]')
plt.ylabel('d_Vv [m/s^2]')
plt.show()

#.. d_Horizontal Velocity
plt.figure()
for i in range(len(UAV)):
    plt.plot(DATA[i].S[:,0], DATA[i].S[:,12], '.-', markersize=4)
plt.grid(True)
plt.xlabel('Time [sec]')
plt.ylabel('d_Vh [m/s^2]')
plt.show()

#.. T Thrust Vector
plt.figure()
for i in range(len(UAV)):
    plt.plot(DATA[i].S[:,0], DATA[i].S[:,16], '.-', markersize=4)
plt.grid(True)
plt.xlabel('Time [sec]')
plt.ylabel('Thrust [N]')
plt.show()




FileNotFoundError: [Errno 2] No such file or directory: './data/matlab.json'